In [1]:
from jupyter_innotater import Innotater
from jupyter_innotater.data import ImageDataWrapper, BoundingBoxDataWrapper, \
                                    MultiClassificationDataWrapper, BinaryClassificationDataWrapper

import numpy as np, os, cv2

### Image Filenames and Bounding Boxes

In [2]:
foodfns = sorted(os.listdir('./foods/'))
targets = np.zeros((len(foodfns), 4), dtype='int') # (x,y,w,h) for each data row

Innotater( ImageDataWrapper(foodfns, path='./foods'), BoundingBoxDataWrapper(targets) )

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [3]:
targets

array([[205, 105,  65,  54],
       [114, 128, 164, 131],
       [170,  65,  74,  76],
       [186, 107,  73,  77],
       [170,  39, 139, 144],
       [137, 129,  80,  76],
       [ 84,  91, 135, 124],
       [225,  39,  94, 116]])

In [4]:
# Write our newly-input bounding box data to disk - will be lost otherwise
import pandas as pd
df = pd.DataFrame(targets, columns=['x','y','w','h'])
df.insert(0,'filename', foodfns)
df.to_csv('./bounding_boxes.csv')

In [5]:
df

,filename,x,y,w,h
0,avocado.jpg,0.0,0.0,0.0,0.0
1,banana.jpg,0.0,0.0,0.0,0.0
2,garlic.jpg,0.0,0.0,0.0,0.0
3,gingerbiscuit.jpg,0.0,0.0,0.0,0.0
4,grapefruit.jpg,0.0,0.0,0.0,0.0
5,lime.jpg,0.0,0.0,0.0,0.0
6,onion.jpg,0.0,0.0,0.0,0.0
7,sweetpotato.jpg,0.0,0.0,0.0,0.0


### Numpy Image Data and Multi-classification

In [3]:
classes = ['vegetable', 'biscuit', 'fruit']
foods = [cv2.imread('./foods/'+f) for f in foodfns]
targets = [0] * len(foodfns)

In [4]:
w2 = Innotater(
        ImageDataWrapper(foods, name='Food'), 
        MultiClassificationDataWrapper(targets, name='FoodType', classes=classes, desc='Food Type')
)
display(w2)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

In [8]:
targets

[0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
# Convert targets from a 1-dim array to one-hot representation - Innotater works with that just as well
onehot_targets = np.squeeze(np.eye(np.array(targets).max()+1)[np.array(targets).reshape(-1)]); onehot_targets

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
Innotater(
    ImageDataWrapper(foods, name='Food'), 
    MultiClassificationDataWrapper(onehot_targets, name='FoodType', classes=classes, desc='Food Type')
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

### Filenames and binary classification

In [25]:
targets = [1,1,0,0,1,1,0,0]
w3 = Innotater(ImageDataWrapper(foodfns, path='./foods'), BinaryClassificationDataWrapper(targets, name='Is Fruit'))
display(w3)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [12]:
targets

[1, 1, 0, 0, 1, 1, 0, 0]

### Image Filenames and Binary Classification plus Bounding Boxes

In [26]:
bboxes = np.zeros((len(foodfns),4))
isfruits = np.ones((len(foodfns),1))

w6 = Innotater(
        ImageDataWrapper(foodfns, name='Food', path='./foods'), 
        [ BinaryClassificationDataWrapper(isfruits, name='Is Fruit'),
          BoundingBoxDataWrapper(bboxes, name='bbs', source='Food', desc='Food Type') ]
)

display(w6)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [27]:
result = np.concatenate([isfruits,bboxes], axis=-1); result

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

### Image versus Image and Binary Classification

In [30]:
targets = np.array([[1,0]] * 5) # One-hot format, defaulting to 0 class
lfoods = foods[:5]
rfoods = lfoods.copy()
rfoods.reverse()

w5 = Innotater([ImageDataWrapper(lfoods, name='Food 1'), ImageDataWrapper(rfoods, name='Food 2')], 
        [BinaryClassificationDataWrapper(targets, name='Are Equal')])
display(w5)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

In [36]:
targets

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

### Two Image Data with named source of Bounding Boxes

In [33]:
foods2 = [f for f in foods]
targets = np.array([[i*10,i*20,i*30,i*40] for i in range(len(foods))])
targets2 = 2 * targets.copy() + 30

w4 = Innotater([ImageDataWrapper(foods, name='Food1'), 
                ImageDataWrapper(foods2, name='Food2')], 
        [
         BoundingBoxDataWrapper(targets, desc='Food Type1', source='Food1'), 
         BoundingBoxDataWrapper(targets2, desc='Food Type2', source='Food2')
        ])
display(w4)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…